In [ ]:
import pandas as pd
from dotenv import load_dotenv

from pydantic import BaseModel
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv(
    dotenv_path="/Users/tomwattley/App/racing-api-project/racing-api-project/libraries/api-helpers/src/api_helpers/.env"
)

from api_helpers.clients import get_local_postgres_client, get_remote_postgres_client

local_pg_client = get_local_postgres_client()
remote_pg_client = get_remote_postgres_client()

local_pg_client.store_data(
    data=remote_pg_client.fetch_data(
        """
                SELECT 
                    todays_betfair_selection_id as selection_id,
                    betfair_win_sp,
                    betfair_place_sp,
                    status,
                    created_at
                FROM live_betting.updated_price_data
                WHERE race_time::date = current_date
                    AND race_time > current_timestamp
        """
    ),
    schema="live_betting",
    table="updated_price_data",
    truncate=True,
)

2025-08-29T17:53:59Z | INFO - Logging configuration initialized with level: INFO


In [ ]:
df = 

2025-08-29T17:57:38Z | INFO - Truncating live_betting.updated_price_data
2025-08-29T17:57:38Z | INFO - Executing query: TRUNCATE TABLE live_betting.updated_price_data
2025-08-29T17:57:38Z | INFO - Query executed. Number of rows affected: -1
2025-08-29T17:57:38Z | INFO - Storing 128 records in live_betting.updated_price_data
